In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import glob
import pydicom
import cv2

In [2]:
train_df = pd.read_csv('../data/train.csv')
desc_df = pd.read_csv('../data/train_series_descriptions.csv')
coor_df = pd.read_csv('../data/train_label_coordinates.csv')

id_path_list = [(x, f'../data/train_images/{x}') for x in os.listdir('../data/train_images')]
id_info_dict = {p[0]: {'folder_path': p[1], 
                       'SeriesInstanceUIDs': os.listdir(p[1]),
                       'SeriesDescriptions': []}
                for p in id_path_list}

for k in id_info_dict:
    for s in id_info_dict[k]['SeriesInstanceUIDs']:
        id_info_dict[k]['SeriesDescriptions'].append(desc_df[(desc_df['study_id'] == int(k)) &
            (desc_df['series_id'] == int(s))]['series_description'].iloc[0])

display(id_info_dict[list(id_info_dict.keys())[0]])

{'folder_path': '../data/train_images/100206310',
 'SeriesInstanceUIDs': ['1012284084', '1792451510', '2092806862'],
 'SeriesDescriptions': ['Axial T2', 'Sagittal T2/STIR', 'Sagittal T1']}

In [3]:
def get_img_dict(patient_id, info_dict):
    patient_info = info_dict[str(patient_id)]
    img_dict = {}
    for idx, i in enumerate(patient_info['SeriesInstanceUIDs']):
        img_dict[i] = {'images': [], 'description': patient_info['SeriesDescriptions'][idx]}
        images = glob.glob(f"{patient_info['folder_path']}/{patient_info['SeriesInstanceUIDs'][idx]}/*.dcm")
        for j in sorted(images, key=lambda x: int(x.replace('\\','/').split('/')[-1].replace('.dcm', ''))):
            img_dict[i]['images'].append({'SOPInstanceUID': j.replace('\\','/').split('/')[-1].replace('.dcm', ''), 'dicom': pydicom.dcmread(j)})
    return img_dict

In [4]:
d1 = get_img_dict(4646740, id_info_dict)
# display(d1[list(d1.keys())[0]]['images'][0]['dicom'])